### 의도 분류 테스트

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


chain = (
    PromptTemplate.from_template(
        """아래 사용자 질문에 대해 '동영상 콘텐츠 실행', '일상 정보', '특정 작업 수행', '기타'에 대한 질문으로 분류하세요.

한 문장 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model="gpt-3.5-turbo-0125")
    | StrOutputParser()
)

# chain.invoke({"question": "Play the movie Titanic."})

chain.invoke({"question": "오늘 서울 기온은 몇 도 인가요? 섭씨온도로 알려주세요."})




'일상 정보'

In [2]:
video_content_chain = PromptTemplate.from_template(
    """당신은 내장되어 있는 영상 정보를 기반으로 타이틀에 매핑된 영상을 재생할 수 있는 능력을 갖춘 에이전트입니다. \
실제로 영상을 재생하지는 않지만, 사용자가 특정 영화 제목을 언급하여 재생하도록 요청하는 경우 해당 제목에 대한 영상을 재생하겠다고 응답하세요. \
다음 질문에 대답하세요:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")


daily_information_chain = PromptTemplate.from_template(
    """당신은 일상 정보에 정통한 에이전트입니다. \
다음 질문에 대답하세요:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")


general_chain = PromptTemplate.from_template(
    """다음 질문에 대답하세요.:

Question: {question}
Answer:"""
) | ChatOpenAI(model="gpt-3.5-turbo-0125")

In [4]:
def route(info):
    if "동영상" in info["topic"].lower():
        return video_content_chain
    elif "일상 정보" in info["topic"].lower():
        return daily_information_chain
    else:
        return general_chain

In [5]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

In [6]:
full_chain.invoke({"question": "영화 타이타닉 재생해줘."})

AIMessage(content='예, 영화 타이타닉을 재생할 수 있습니다. 잠시만 기다려주세요. 영상을 재생합니다.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 163, 'total_tokens': 204}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7ba55cc2-d08f-43b0-a9f6-21177b73ebde-0')

In [7]:
full_chain.invoke({"question": "오늘 서울 기온은 몇 도야? 섭씨로 대답해주고 오늘 날짜 정보도 포함해서 대답해줘."})

AIMessage(content='서울의 오늘 기온은 섭씨 25도이며, 오늘은 2021년 10월 14일 입니다.', response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 86, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1da01f4f-cede-4411-b37a-6a8790c5017c-0')